# Deeper Into SQL
We have work with databases and tables that have already been created, in this lesson we will learn how to create tables and link them together. 

##  Normalized Design
In a normalized database, the relationships among the tables match the relationships that are really there among the data.

1. Every row has the same number of columns.
2. There is a unique key and everything in a row says something about the key.
3. Facts that don't relate to the key belong in different tables
4. Tables should't imply relationships that don't exist

Let look at an example of rule number 1
![](img/normal.PNG)

If we have two value with the same meaning for a given key we are going to have make them separate rows like below.  
![](img/normal2.PNG)

Let look at an example of rule number 2
![](img/normal3.PNG)
Part num is the unique key that will define by with its description of color, material and shape.

Let look at an example of rule number 3
![](img/normal4.PNG)
To normalize the table we have we would want to split out the mapping from location to address from the rest of the table like the example below. This allows us to talk about a location without talking about a particular item. 

![](img/normal5.PNG)

Let look at an example of rule number 4

![](img/normal7.PNG)

Here Annabel knows technology; Databases, Linux and Data Science. She also knows the language English, French, Arabic. In this example Databases have nothing to do with English and Linux has nothing to do with french but these rows make it look like they do. To corrected this we want to make separate tables for people and there technology and people and their languages like below. 

![](img/normal8.PNG)

## Create Table and Types
We have worked out queries before so lets see an examples of how to create a table.
create table tablename(column1 type, column2 type);


![](img/type.PNG)

Some systems suchs as PostgreSQL have abbreciations for longer types names such as timestamptz in place of timestamp with time zone.

![](img/type2.PNG)

Normally if you are creating a database for a new application you would like to do that during installation.

There are occasions where you'd want to create tables from within your application's code. For instance, this can be a handy way to have it set itself up when you first run it. (That's what your browser does when it creates your cookies file, for instance.)

Lets create a table in using vagrant and psql. First we would like to log into vagrant then , if you need help please refer to [part 3](Intro-Relational-DB.ipynb).

Run the <code>psql</code> command and lets get started!

![](img/psql.PNG)

First we will create a database called fishies. To do that we run <code>CREATE DATABASE fishies;</code> in psql. You should get a prompt stating "CREATE DATABASE". Should have something similar to the image below

![](img/psql2.PNG)

If would like to explore more please the [documentation](https://www.postgresql.org/docs/9.4/sql-createdatabase.html).

To remove a database we would use the [Drop Database](https://www.postgresql.org/docs/9.4/sql-dropdatabase.html) command.
Connect to it with <code>\c fishies</code>, or by exiting psql and running <code>psql fishies</code>.

In the new database, create a table that has two columns: a text column and a serial column.

![](img/psql3.PNG)

Lets now [create a table](http://www.postgresql.org/docs/9.4/static/sql-createtable.html) called fishies which has two columns: a text column called name and a serial column called id. Run the command <code>CREATE TABLE fishies (name text, id serial);</code>. Again you would be prompted with "CREATE TABLE".

![](img/psql4.PNG)

Let's add 3 items to get a feel for what the database may be doing. Let's do 3 separate inserts 

```
insert into fishies values ('Nemo', 4);
insert into fishies values ('Marlin');
insert into fishies values ('Dory');
```

![](img/psql5.PNG)

What do you think our table looks like? Let's check it using the <code>SELECT</code> keyword. Run <code>SELECT * FROM fishies;</code>

![](img/psql6.PNG)

Looks like even though we didn't fill in a value for the id of Marlin and Dory, PSQL filled in id for us. If you would like more info on the serial type please check the [documentation](http://www.postgresql.org/docs/9.4/static/datatype-numeric.html) 

## Declaring Primary Key
At first we may think that the name of city might be a good choice as that is often how we id a city in real life, byt the problem that there may be other cities with the same name and primary keys must be unique. In the example below we see that there are 5 springfields in Wisconsin. If we want to make a primary key for places we would need to use something suchs at coordinates or zipcode. 

![](img/primary.PNG)

In SQL we have to must assign a primary key if we want to explicitly relate other tables. Here are two examples of how we would do it. If we have a single column primary key we just add <code>primary key</code> after the columns type when we create the table. The primary key should be before all the other column in the table

![](img/primary2.PNG)

In the case that we have a multicolumn key like in the case of using postal code and country we would put it after all the columns in the table. In the case of postal codes, we would also need countries as the some country may share the same postal code for different locations.

![](img/primary3.PNG)

If we try to insert a duplicate primary key into a table our database will signal an error. 

## Declaring Relationships
When we assign type to a column we restrict the values that can be put into them. In the example below if we try to insert the values "pony" and "pies" the database will pick up that "pies" is not an integer.

![](img/relation.PNG)

We may want to put additional restriction on to a database. Imagine that we want have two tables; one for products with the primary key called sku and a second table for sales we have made. We expect that every sku in the sales table will refer to one in the products table. Say we inserted a sku in the sales table that wasn't in the products table, it wouldn't make any sense as we could not sell a product that we don't have. The database can catch errors like this but only if we tell it to.

![](img/relation2.PNG)

We can tell our database to watch our sku number that are not register in our products table by create a table with a command like the one below. 

![](img/relation3.PNG)

References provide referential integrity - columns that are supposed to refer to each other are guaranteed to do so.

In database terminology, a column with a references constraint on it is also called a foreign key. The definition of a foreign key is a column or set of columns in one table, that uniquely identifies rows in another table. It possible for a table to have two or more foreign keys and is actually really common. Below we have an example of this where the grades table references the student and courses table first to make sure that the values exist in those tables. 

![](img/relation4.PNG)

## Self Join
We have used join to to derive two or more existing tables but there are time when you may want a join an existing table to itself. For instance to find pairs of entries that have something in common. Doing these operations in a database is generally a lot after than pulling the data into our python code and standing it there. Below we have a table of students living on campus.

![](img/relation5.PNG)

We would like to see which students are roomates. Running the query below will get us almost there

```
select a.id, b.id, a.building, a.room
       from residences as a, residences as b
 where a.building = b.building
   and a.room = b.room
 order by a.building, a.room;
```

We seem to be missing one part. The query above will have two copies of the residences table and will return all matching pairs for building and room number. Its seem that since the table is a copy all students will match with themselves as roommates but also that there will be two copies of it. The query below fixes this problem by adding the condition <code>a.id < b.id </code>. This condition makes sure no one can be roomates with themselves and gets rid of any change of duplicate when matching roommates.

```
select a.id, b.id, a.building, a.room
       from residences as a, residences as b
 where a.building = b.building
   and a.room = b.room 
   and a.id < b.id
 order by a.building, a.room;
```
                  
## Subqueries
So far we have only worked on queries on a table or multiple tables but we will now talk about subqueries. We can use a select call from a results table. This allows us to do a task that would maybe taken 2 queries and shorten it into a single query.
                  
![](img/select.PNG)

Below we have an example of us running a subquery and naming it maxes even though we never use it in our query on the average bigscore. PSQL requires us to give the subquery a name.
                  
![](img/select2.PNG)
          
## Views
A view is a select query stored in the database in a way that lets you use it like a table. They are really useful with aggregation say we like to check the count for how many student are in a course. It would be smart for us to save this in a view as we refer to it often.